In [23]:
from pyspark.sql import SparkSession, SQLContext
from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import array, udf, col, explode, concat_ws, collect_list, sort_array, count, size, coalesce, expr, regexp_replace
spark= SparkSession.builder.config('spark.ui.port','4040').getOrCreate()
spark.conf.set('spark.sql.caseSensitive', 'True') 

ImportError: cannot import name 'selectExpr'

In [2]:
WoS = spark.read.format("parquet").load("/WoSraw_2020_all/parquet/part-00826-5476e87f-146e-4916-913b-522638c0d728-c000.snappy.parquet")

In [3]:
WoS.printSchema()

root
 |-- UID: string (nullable = true)
 |-- _r_id_disclaimer: string (nullable = true)
 |-- dynamic_data: struct (nullable = true)
 |    |-- cluster_related: struct (nullable = true)
 |    |    |-- identifiers: struct (nullable = true)
 |    |    |    |-- identifier: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |    |-- _type: string (nullable = true)
 |    |    |    |    |    |-- _value: string (nullable = true)
 |    |-- ic_related: struct (nullable = true)
 |    |    |-- oases: struct (nullable = true)
 |    |    |    |-- _count: long (nullable = true)
 |    |    |    |-- _is_OA: string (nullable = true)
 |    |    |    |-- oas: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |    |-- _VALUE: string (nullable = true)
 |    |    |    |    |    |-- _type: string (nullable = true)
 |-- static_data: str

In [38]:
wosEdition = WoS.select(WoS.UID.alias("wosID"),
                        WoS.static_data.summary.EWUID.edition[0]._value.alias('Edition')
                       )

wosEdition.show(5, truncate = False)
#wosEdition.printSchema()

+-------------------+--------+
|wosID              |Edition |
+-------------------+--------+
|WOS:000591598700044|WOS.ESCI|
|WOS:000575759200001|WOS.ESCI|
|WOS:000596077600005|WOS.ESCI|
|WOS:000593375100004|WOS.ESCI|
|WOS:000596884600012|WOS.ESCI|
+-------------------+--------+
only showing top 5 rows



In [53]:
wosCat = WoS.select(WoS.UID.alias("wosID"),
                    WoS.static_data.fullrecord_metadata.category_info.headings.heading[0].alias("heading"),
                    WoS.static_data.fullrecord_metadata.category_info.subheadings.subheading[0].alias("subheading"),
                    WoS.static_data.fullrecord_metadata.category_info.subjects.subject["_VALUE"][0].alias("subject")
                    )

wosCat.show(5, truncate = False)

#wosCat.printSchema()

+-------------------+--------------------+---------------------------+-----------------------------------------+
|wosID              |heading             |subheading                 |subject                                  |
+-------------------+--------------------+---------------------------+-----------------------------------------+
|WOS:000591598700044|Science & Technology|Technology                 |Metallurgy & Metallurgical Engineering   |
|WOS:000575759200001|Social Sciences     |null                       |Hospitality, Leisure, Sport & Tourism    |
|WOS:000596077600005|Science & Technology|Technology                 |Engineering, Environmental               |
|WOS:000593375100004|Science & Technology|Technology                 |Computer Science, Artificial Intelligence|
|WOS:000596884600012|Science & Technology|Life Sciences & Biomedicine|Nursing                                  |
+-------------------+--------------------+---------------------------+--------------------------